# The Cancer Gene Atlas Cancer (TCGA) - Kidney Renal Clear Cell Carcinoma (KIRC)

Author: Moshe Silverstein <br>
Date: 7-17 <br>
Data Source: https://cancergenome.nih.gov/

In [1]:
import sys, datetime, os
import numpy as np
import pandas as pd
import importlib
import untility_functions as uf
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter

In [2]:
import seaborn as sns
sns.set(color_codes=True)
np.random.seed(sum(map(ord, "distributions")))

In [3]:
from clustergrammer_widget import *
net = Network(clustergrammer_widget)

In [4]:
importlib.reload(uf)

<module 'untility_functions' from '/Users/moshesilverstein/Documents/Harmonizome/TCGA/untility_functions.py'>

# Versions Of Modules In Use

In [5]:
%load_ext version_information
%version_information numpy, pandas, clustergrammer_widget, seaborn 

Software versions
Python 3.5.2 64bit [GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)]
IPython 5.3.0
OS Darwin 17.5.0 x86_64 i386 64bit
numpy 1.13.1
pandas 0.21.0
clustergrammer_widget 1.9.0
seaborn 0.7.1
Fri Apr 27 13:44:48 2018 EDT

# Functions

In [6]:
def getGenes(inputDF):
    
    inputDF.reset_index(inplace=True)
    
    inputDF.rename(columns={'index':'Gene Symbol'}, inplace=True)

    inputDF.set_index('Gene Symbol', inplace=True)

    lst = []

    for index in  inputDF.index:
        lst.append(index.split('|')[5])

    inputDF.index = lst

In [7]:
def getSampleID(inputDF, metaDF):
    
    lst = []
    
    for i,col in enumerate(inputDF.columns):

        progressPercent = ((i+1)/len(inputDF.columns))*100

        sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(inputDF.columns)))
        sys.stdout.flush()

        ID = ('-').join(metaDF.loc[col, 'a_AliquotBarcode'].split('-')[0:3])
        
        ID = ID+'-'+metaDF.loc[col, 'a_AliquotBarcode'].split('-')[3][0:2]

        lst.append(ID)

        
    inputDF.columns = lst

In [8]:
def getSampleData(inputDF, metaDF, SampleTypes):
    
    lst = []

    for i,col in enumerate(inputDF.columns):

        progressPercent = ((i+1)/len(inputDF.columns))*100

        sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(inputDF.columns)))
        sys.stdout.flush()
        
#         ID = ('-').join(meta.loc[col, 'a_AliquotBarcode'].split('-')[0:3])
        ID = ('-').join(col.split('-')[0:3])
        
        donor = 'Donor:'+ID
        
#         if '01' in str(col.split('-')[3]):
#             sampleType = 'Primary Solid Tumor'
#         elif '11' in str(col.split('-')[3]):
#             sampleType= 'Solid Tissue Normal'
        
        if ID in metaDF.index:
            info = (donor, 
                'Age:'+str(metaDF.loc[ID, 'age_at_initial_pathologic_diagnosis']),
                'Gender:'+str(metaDF.loc[ID, 'gender']),    
                'SampleType:'+SampleTypes[str(col.split('-')[3])],
                'Stage:'+str(metaDF.loc[ID, 'pathologic_stage']),
                'Histology:'+str(metaDF.loc[ID, 'histological_type']),
                'Tissue:'+str(metaDF.loc[ID, 'tumor_tissue_site']), 
                'Tumor:'+str(metaDF.loc[ID, 'tumor_type']),    
                'Grade'+str(metaDF.loc[ID, 'neoplasm_histologic_grade'])
#                 'SampleType:'+str(ID.split('-')[3])
               )
#             for col in metaDF.columns:
#                 if len(metaDF[col].dropna()) >= .5*len(metaDF[col]) and len(metaDF[col].unique()) > 2 and len(metaDF[col].unique()) < len(metaDF[col]) and 'age' not in col:
#                     info = info + (str(col)+':'+str(metaDF.loc[ID, col]),)
        lst.append(info)

        
    inputDF.columns = lst

In [9]:
SampleTypes ={
    '01':'Primary Solid Tumor',
    '02':'Recurrent Solid Tumor',
    '03':'Primary Blood Derived Cancer - Peripheral Blood',
    '04':'Recurrent Blood Derived Cancer - Bone Marrow',
    '05':'Additional - New Primary',
    '06':'Metastatic',
    '07':'Additional Metastatic',
    '08':'Human Tumor Original Cells',
    '09':'Primary Blood Derived Cancer - Bone Marrow',
    '10':'Blood Derived Normal',
    '11':'Solid Tissue Normal',
    '12':'Buccal Cell Normal',
    '13':'EBV Immortalized Normal',
    '14':'Bone Marrow Normal',
    '15':'sample type 15',
    '16':'sample type 16',
    '20':'Control Analyte',
    '40':'Recurrent Blood Derived Cancer - Peripheral Blood',
    '50':'Cell Lines',
    '60':'Primary Xenograft Tissue',
    '61':'Cell Line Derived Xenograft Tissue',
    '99':'sample type 99'
             }

# Path to Output Files

In [10]:
path = '/Users/moshesilverstein/Documents/Harmonizome/TCGA/Output/'

# Load Meta Data

In [11]:
meta = pd.read_csv('Input/TCGA_Metadata.csv', index_col=9)

In [12]:
meta.head()

,a_AliquotBarcode,b_disease,a_objectSizeBytes,b_uploaded,b_state,b_library_type,b_center,b_assembly,a_GCSobject
a_CGHubAnalysisID,,,,,,,,,
3a8e6a74-137f-468f-8987-fa0acdde2836,TCGA-CS-6188-01A-11R-1896-07,LGG,6354301047,8/14/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
c63d44f1-c9ad-4391-b1aa-0feb3713441b,TCGA-EO-A22Y-01A-11R-A180-07,UCEC,5497637066,8/29/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
21912089-1e42-4bcc-9ad9-fe9a9b88fb09,TCGA-94-A5I4-01A-11R-A26W-07,LUSC,4600317850,8/29/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
6b86e544-0372-434a-9f28-6fa4075dd228,TCGA-N5-A4RV-01A-21R-A28V-07,UCS,4964387768,8/30/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
30518eb4-2783-4988-bcd4-36a1a8bb4dfa,TCGA-BK-A56F-01A-32R-A27V-07,UCEC,5668623460,8/31/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...


In [13]:
meta.shape

(11373, 9)

# Load Sample MetaData

In [14]:
sample_meta = pd.read_csv('Input/metadata_Kidney renal clear cell carcinoma [KIRC]', sep='\t', index_col=1)

In [15]:
sample_meta.head()

,Unnamed: 0,Study,Project,ParticipantUUID,TSSCode,age_at_initial_pathologic_diagnosis,anatomic_neoplasm_subdivision,batch_number,bcr,clinical_M,...,BMI,age_began_smoking_in_years,h_pylori_infection,other_dx,other_malignancy_anatomic_site,other_malignancy_histological_type,other_malignancy_malignancy_type,stopped_smoking_year,venous_invasion,year_of_tobacco_smoking_onset
ParticipantBarcode,,,,,,,,,,,,,,,,,,,,,
TCGA-3Z-A93Z,8294,KIRC,TCGA,2B1DEA0A-6D55-4FDD-9C1C-0D9FBE03BD78,3Z,69.0,NaN,387,Nationwide Children's Hospital,M0,...,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-6D-AA2E,8295,KIRC,TCGA,D3B47E53-6F40-4FC8-B5A4-CBE548A770A9,6D,68.0,NaN,387,Nationwide Children's Hospital,MX,...,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-A3-3365,8296,KIRC,TCGA,136c95cd-febd-43f0-af35-125d3321a145,A3,46.0,NaN,32,Nationwide Children's Hospital,NaN,...,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-A3-A8OX,8297,KIRC,TCGA,C33D0D12-A744-4A9A-B3F7-E28DA6A109AB,A3,65.0,NaN,387,Nationwide Children's Hospital,M0,...,NaN,NaN,NaN,"Yes, History of Synchronous/Bilateral Malignancy",Breast,"Other, specify",Synchronous Malignancy,NaN,NaN,NaN
TCGA-A3-3351,8298,KIRC,TCGA,ee7011e7-f692-410a-ba35-efe1ec75cf8e,A3,42.0,NaN,50,Nationwide Children's Hospital,NaN,...,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
sample_meta.shape

(537, 70)

# Load Data

In [17]:
matrix = pd.read_csv('Input/TCGA_KIRC_tpm.tsv', sep='\t', index_col=0)

In [18]:
matrix.head()

,9e7c8587-4a7e-4aec-aee8-9796933ea560,1e65f3bc-fa61-4083-b0e4-86a97ada3aae,953473f4-9927-4fd4-bfee-8f5908638cc7,df66c814-1407-4899-a6e3-9c2a928e6ea0,8cc55b0c-b386-4178-ba34-41b481fe4470,2bdf69ff-b6fe-48c8-ac05-74ca195c6e1d,63e92d5f-1d0a-4fda-a8e8-3a7fbaa2635b,120ff28b-5864-40c4-8bad-a0dfc5deccd5,4c6e22a5-4455-4893-89a4-cfa7ac74b634,87c1d789-9482-442a-8b42-97bb479060e3,...,b72f5afa-7187-4246-a3dc-e9ec04764f8b,9bae55b1-5a27-4a9f-b376-ad1ab4fc4ee2,6b124b2e-dce9-48ef-bedb-7a2da9cd2ff8,3b4be095-1d76-49de-8357-f932ba5d55bb,e644640b-78d3-460a-83fa-ea475620a3f0,01f8736a-85bc-498d-91c6-1c053c479512,e40e21de-f116-4eb1-87af-3b16797ec3b2,19de7074-f195-47f6-81fa-6d3bf01c1685,539cc430-1233-42e4-a07f-18cbf4a10d16,0153168a-39ce-47d6-b7f5-ac1e0f312245
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,131.46900,119.52300,111.32400,109.207000,135.791000,100.2100,165.68300,97.81970,101.51700,103.83500,...,189.80000,132.076000,105.54200,153.906000,108.729000,151.7070,135.462000,99.334200,144.17800,119.98000
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,72.44600,55.21540,56.85380,69.043700,47.684900,60.5701,51.00020,41.50070,61.39290,67.55030,...,5.34424,41.404600,44.61230,42.531000,36.538400,52.7045,39.144800,23.720600,30.16480,31.06460
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,28.53610,22.34360,19.12880,17.753100,22.181000,18.9952,17.23290,33.69740,12.60810,18.90270,...,14.63530,23.868700,48.59210,39.508100,26.315200,21.9785,23.691100,31.972400,25.48650,13.46400
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,52.17980,28.05210,25.25420,30.200000,26.151600,29.8227,28.69310,58.91650,26.76780,38.46310,...,14.97550,26.360100,23.99270,40.887300,20.092700,29.8759,26.511100,13.060100,20.56090,23.52400
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,1.76568,1.45036,3.29564,0.993345,0.906265,1.0326,2.15756,8.53379,2.18625,6.59523,...,0.00000,0.894234,2.17009,0.694892,0.232657,3.3030,0.322567,0.364854,1.26402,7.22618


In [19]:
matrix.shape

(199169, 618)

# Get Only Protein Coding Genes

In [20]:
lst = []

for i,index in enumerate(matrix.index):
    
    progressPercent = ((i+1)/len(matrix.index)*100)

    sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(matrix.index)))
    sys.stdout.flush()
                                 
    lst.append(index.split('|')[7])    
                       
matrix['Gene Type'] = lst

matrix = matrix[matrix['Gene Type'] == 'protein_coding']

matrix.drop('Gene Type', axis=1, inplace=True)

In [21]:
matrix.head()

,9e7c8587-4a7e-4aec-aee8-9796933ea560,1e65f3bc-fa61-4083-b0e4-86a97ada3aae,953473f4-9927-4fd4-bfee-8f5908638cc7,df66c814-1407-4899-a6e3-9c2a928e6ea0,8cc55b0c-b386-4178-ba34-41b481fe4470,2bdf69ff-b6fe-48c8-ac05-74ca195c6e1d,63e92d5f-1d0a-4fda-a8e8-3a7fbaa2635b,120ff28b-5864-40c4-8bad-a0dfc5deccd5,4c6e22a5-4455-4893-89a4-cfa7ac74b634,87c1d789-9482-442a-8b42-97bb479060e3,...,b72f5afa-7187-4246-a3dc-e9ec04764f8b,9bae55b1-5a27-4a9f-b376-ad1ab4fc4ee2,6b124b2e-dce9-48ef-bedb-7a2da9cd2ff8,3b4be095-1d76-49de-8357-f932ba5d55bb,e644640b-78d3-460a-83fa-ea475620a3f0,01f8736a-85bc-498d-91c6-1c053c479512,e40e21de-f116-4eb1-87af-3b16797ec3b2,19de7074-f195-47f6-81fa-6d3bf01c1685,539cc430-1233-42e4-a07f-18cbf4a10d16,0153168a-39ce-47d6-b7f5-ac1e0f312245
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,131.46900,119.52300,111.32400,109.207000,135.791000,100.2100,165.68300,97.81970,101.51700,103.83500,...,189.80000,132.076000,105.54200,153.906000,108.729000,151.7070,135.462000,99.334200,144.17800,119.98000
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,72.44600,55.21540,56.85380,69.043700,47.684900,60.5701,51.00020,41.50070,61.39290,67.55030,...,5.34424,41.404600,44.61230,42.531000,36.538400,52.7045,39.144800,23.720600,30.16480,31.06460
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,28.53610,22.34360,19.12880,17.753100,22.181000,18.9952,17.23290,33.69740,12.60810,18.90270,...,14.63530,23.868700,48.59210,39.508100,26.315200,21.9785,23.691100,31.972400,25.48650,13.46400
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,52.17980,28.05210,25.25420,30.200000,26.151600,29.8227,28.69310,58.91650,26.76780,38.46310,...,14.97550,26.360100,23.99270,40.887300,20.092700,29.8759,26.511100,13.060100,20.56090,23.52400
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,1.76568,1.45036,3.29564,0.993345,0.906265,1.0326,2.15756,8.53379,2.18625,6.59523,...,0.00000,0.894234,2.17009,0.694892,0.232657,3.3030,0.322567,0.364854,1.26402,7.22618


In [22]:
matrix.shape

(79930, 618)

# Map Sample Meta Data to Sample ID

In [23]:
getSampleID(matrix, meta)

In [24]:
matrix.head()

,TCGA-CZ-5989-11,TCGA-BP-5186-01,TCGA-EU-5904-01,TCGA-EU-5906-01,TCGA-BP-5184-01,TCGA-EU-5905-01,TCGA-AK-3444-01,TCGA-CW-5585-11,TCGA-A3-3376-01,TCGA-B8-5552-01,...,TCGA-G6-A8L7-01,TCGA-A3-A8OU-01,TCGA-6D-AA2E-01,TCGA-A3-A8OV-01,TCGA-A3-A8OW-01,TCGA-A3-A8CQ-01,TCGA-B8-A7U6-01,TCGA-B8-A54D-01,TCGA-MM-A563-01,TCGA-B8-A8YJ-01
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,131.46900,119.52300,111.32400,109.207000,135.791000,100.2100,165.68300,97.81970,101.51700,103.83500,...,189.80000,132.076000,105.54200,153.906000,108.729000,151.7070,135.462000,99.334200,144.17800,119.98000
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,72.44600,55.21540,56.85380,69.043700,47.684900,60.5701,51.00020,41.50070,61.39290,67.55030,...,5.34424,41.404600,44.61230,42.531000,36.538400,52.7045,39.144800,23.720600,30.16480,31.06460
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,28.53610,22.34360,19.12880,17.753100,22.181000,18.9952,17.23290,33.69740,12.60810,18.90270,...,14.63530,23.868700,48.59210,39.508100,26.315200,21.9785,23.691100,31.972400,25.48650,13.46400
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,52.17980,28.05210,25.25420,30.200000,26.151600,29.8227,28.69310,58.91650,26.76780,38.46310,...,14.97550,26.360100,23.99270,40.887300,20.092700,29.8759,26.511100,13.060100,20.56090,23.52400
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,1.76568,1.45036,3.29564,0.993345,0.906265,1.0326,2.15756,8.53379,2.18625,6.59523,...,0.00000,0.894234,2.17009,0.694892,0.232657,3.3030,0.322567,0.364854,1.26402,7.22618


# Map Gene Symbol to Gene ID

In [25]:
getGenes(matrix)

In [26]:
matrix.head()

,TCGA-CZ-5989-11,TCGA-BP-5186-01,TCGA-EU-5904-01,TCGA-EU-5906-01,TCGA-BP-5184-01,TCGA-EU-5905-01,TCGA-AK-3444-01,TCGA-CW-5585-11,TCGA-A3-3376-01,TCGA-B8-5552-01,...,TCGA-G6-A8L7-01,TCGA-A3-A8OU-01,TCGA-6D-AA2E-01,TCGA-A3-A8OV-01,TCGA-A3-A8OW-01,TCGA-A3-A8CQ-01,TCGA-B8-A7U6-01,TCGA-B8-A54D-01,TCGA-MM-A563-01,TCGA-B8-A8YJ-01
ARF5,131.46900,119.52300,111.32400,109.207000,135.791000,100.2100,165.68300,97.81970,101.51700,103.83500,...,189.80000,132.076000,105.54200,153.906000,108.729000,151.7070,135.462000,99.334200,144.17800,119.98000
M6PR,72.44600,55.21540,56.85380,69.043700,47.684900,60.5701,51.00020,41.50070,61.39290,67.55030,...,5.34424,41.404600,44.61230,42.531000,36.538400,52.7045,39.144800,23.720600,30.16480,31.06460
ESRRA,28.53610,22.34360,19.12880,17.753100,22.181000,18.9952,17.23290,33.69740,12.60810,18.90270,...,14.63530,23.868700,48.59210,39.508100,26.315200,21.9785,23.691100,31.972400,25.48650,13.46400
FKBP4,52.17980,28.05210,25.25420,30.200000,26.151600,29.8227,28.69310,58.91650,26.76780,38.46310,...,14.97550,26.360100,23.99270,40.887300,20.092700,29.8759,26.511100,13.060100,20.56090,23.52400
CYP26B1,1.76568,1.45036,3.29564,0.993345,0.906265,1.0326,2.15756,8.53379,2.18625,6.59523,...,0.00000,0.894234,2.17009,0.694892,0.232657,3.3030,0.322567,0.364854,1.26402,7.22618


# Histogram of First Sample

In [27]:
sns.distplot(matrix.iloc[:, 0]);

# Histogram of First Gene

In [28]:
sns.distplot(matrix.iloc[0, :]);

# Save Unfiltered Matrix to File

In [29]:
filename = path+'tcga_kirc_matrix_unfilltered_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
matrix.to_csv(filename, sep='\t', compression='gzip')

# Filtered

In [30]:
normalized_matrix = matrix.copy()

In [31]:
normalized_matrix = normalized_matrix.groupby(level=0).max()

# Remove any data with more than 95% missing and imput remaining mssing values to matrix mean

In [32]:
normalized_matrix = uf.removeAndImpute(normalized_matrix)

In [33]:
normalized_matrix.head()

,TCGA-CZ-5989-11,TCGA-BP-5186-01,TCGA-EU-5904-01,TCGA-EU-5906-01,TCGA-BP-5184-01,TCGA-EU-5905-01,TCGA-AK-3444-01,TCGA-CW-5585-11,TCGA-A3-3376-01,TCGA-B8-5552-01,...,TCGA-G6-A8L7-01,TCGA-A3-A8OU-01,TCGA-6D-AA2E-01,TCGA-A3-A8OV-01,TCGA-A3-A8OW-01,TCGA-A3-A8CQ-01,TCGA-B8-A7U6-01,TCGA-B8-A54D-01,TCGA-MM-A563-01,TCGA-B8-A8YJ-01
A1BG,1.412427,0.020520,1.412427,1.412427,1.412427,1.412427,1.412427,1.412427,1.412427,1.412427,...,1.412427,1.412427,0.039229,1.412427,1.412427,1.412427,1.412427,1.412427,1.412427,1.412427
A1CF,0.335945,10.124900,14.375400,22.775200,7.821610,4.807120,13.463800,7.595370,4.920660,6.097610,...,0.021686,5.444300,0.642040,5.756610,10.341400,7.809060,2.568220,2.734430,1.948260,1.268970
A2M,560.079000,523.509000,950.864000,859.691000,480.333000,477.709000,329.837000,367.060000,1085.560000,1128.830000,...,53.647100,410.485000,115.480000,228.600000,803.017000,489.170000,392.518000,182.729000,429.327000,1225.130000
A2ML1,0.141185,0.068294,0.170120,0.082989,0.046049,0.069423,0.101530,0.063351,0.165947,0.087483,...,0.080108,0.204305,0.179825,0.105629,0.137201,0.100491,0.107099,0.313054,0.059625,0.112725
A3GALT2,0.023979,0.045246,0.023789,0.035036,0.096744,0.074899,0.009363,0.096744,0.101544,0.066869,...,0.027793,0.173970,0.096744,0.050932,0.053394,0.030438,0.096744,0.144540,0.096744,0.116545


In [34]:
normalized_matrix.shape

(19188, 618)

# logCPM Transform

Raw counts were normalized to log10-Counts Per Million (logCPM) by dividing each column by the total sum of its counts, multiplying it by 106, followed by the application of a log10-transform.

In [35]:
samples = normalized_matrix.columns.tolist()

In [36]:
normalized_matrix.columns = np.arange(0, len(normalized_matrix.columns))

In [37]:
for col in normalized_matrix.columns:

    total = normalized_matrix[col].sum()
    
    normalized_matrix[col] = normalized_matrix[col].apply(lambda x: np.log10(((x)/total)*10**6))

In [38]:
normalized_matrix.columns = samples

In [39]:
normalized_matrix.head()

,TCGA-CZ-5989-11,TCGA-BP-5186-01,TCGA-EU-5904-01,TCGA-EU-5906-01,TCGA-BP-5184-01,TCGA-EU-5905-01,TCGA-AK-3444-01,TCGA-CW-5585-11,TCGA-A3-3376-01,TCGA-B8-5552-01,...,TCGA-G6-A8L7-01,TCGA-A3-A8OU-01,TCGA-6D-AA2E-01,TCGA-A3-A8OV-01,TCGA-A3-A8OW-01,TCGA-A3-A8CQ-01,TCGA-B8-A7U6-01,TCGA-B8-A54D-01,TCGA-MM-A563-01,TCGA-B8-A8YJ-01
A1BG,0.318576,-1.503786,0.324747,0.346483,0.320254,0.337284,0.309221,0.288299,0.357464,0.339727,...,0.351441,0.363628,-1.236442,0.336222,0.356263,0.376301,0.350984,0.337908,0.370861,0.389832
A1CF,-0.305122,1.189432,1.332401,1.553979,1.063585,0.869203,1.288422,1.018882,0.899521,0.974921,...,-1.462342,0.949604,-0.022487,0.946423,1.220876,1.118934,0.610650,0.624809,0.510542,0.343318
A2M,2.916859,2.902965,3.152899,3.130859,2.851831,2.866482,2.677554,2.703070,3.243151,3.242390,...,1.931021,2.826959,2.232457,2.545333,3.111021,2.915795,2.794877,2.449749,2.853683,3.328049
A2ML1,-0.681602,-0.981578,-0.594464,-0.884462,-1.166495,-0.971179,-0.834151,-1.059914,-0.572533,-0.868316,...,-0.894848,-0.476059,-0.575199,-0.789961,-0.656346,-0.771537,-0.769197,-0.316439,-1.003673,-0.708113
A3GALT2,-1.451556,-1.160377,-1.448852,-1.258967,-0.844088,-0.938205,-1.869316,-0.876043,-0.785848,-0.985012,...,-1.354594,-0.545863,-0.844425,-1.106754,-1.066211,-1.290254,-0.813358,-0.652070,-0.793481,-0.693640


# Normalize Matrix (Quantile Normalize the matrix for the columns)

In [40]:
normalized_matrix = uf.quantileNormalize(normalized_matrix)

In [41]:
normalized_matrix.head()

,TCGA-CZ-5989-11,TCGA-BP-5186-01,TCGA-EU-5904-01,TCGA-EU-5906-01,TCGA-BP-5184-01,TCGA-EU-5905-01,TCGA-AK-3444-01,TCGA-CW-5585-11,TCGA-A3-3376-01,TCGA-B8-5552-01,...,TCGA-G6-A8L7-01,TCGA-A3-A8OU-01,TCGA-6D-AA2E-01,TCGA-A3-A8OV-01,TCGA-A3-A8OW-01,TCGA-A3-A8CQ-01,TCGA-B8-A7U6-01,TCGA-B8-A54D-01,TCGA-MM-A563-01,TCGA-B8-A8YJ-01
A1BG,0.138005,-1.529053,0.249674,0.199476,0.313507,0.226442,0.366700,0.335811,0.131895,0.147895,...,0.655039,0.249897,-1.235209,0.341930,0.205781,0.170738,0.304789,0.312030,0.227119,0.189337
A1CF,-0.430574,1.083407,1.222690,1.381431,1.043527,0.786778,1.250382,1.044195,0.683907,0.775350,...,-1.454128,0.846861,0.040673,0.929158,1.084058,0.908818,0.548378,0.617534,0.391821,0.135808
A2M,2.926865,2.869542,3.075276,3.065796,2.803780,2.778523,2.648343,2.757415,3.175658,3.186022,...,2.044684,2.735405,2.228244,2.494305,3.042344,2.865777,2.675126,2.572708,2.810698,3.270417
A2ML1,-0.790554,-0.987101,-0.584151,-0.866340,-1.138539,-1.028374,-0.668351,-0.971244,-0.713428,-0.918050,...,-0.804051,-0.576019,-0.531416,-0.750764,-0.749950,-0.886618,-0.817314,-0.364734,-1.085721,-0.883099
A3GALT2,-1.530146,-1.166672,-1.457701,-1.240070,-0.802395,-0.996056,-1.624992,-0.784838,-0.886618,-1.028095,...,-1.347714,-0.634223,-0.806228,-1.078133,-1.134831,-1.401268,-0.860438,-0.713152,-0.882281,-0.868820


# Normalize Matrix (z-score the rows)

In [42]:
uf.zscore(normalized_matrix, 'row')

In [43]:
normalized_matrix.head()

,TCGA-CZ-5989-11,TCGA-BP-5186-01,TCGA-EU-5904-01,TCGA-EU-5906-01,TCGA-BP-5184-01,TCGA-EU-5905-01,TCGA-AK-3444-01,TCGA-CW-5585-11,TCGA-A3-3376-01,TCGA-B8-5552-01,...,TCGA-G6-A8L7-01,TCGA-A3-A8OU-01,TCGA-6D-AA2E-01,TCGA-A3-A8OV-01,TCGA-A3-A8OW-01,TCGA-A3-A8CQ-01,TCGA-B8-A7U6-01,TCGA-B8-A54D-01,TCGA-MM-A563-01,TCGA-B8-A8YJ-01
A1BG,-1.228116,-13.558108,-0.402184,-0.773458,0.069947,-0.574010,0.463374,0.234912,-1.273305,-1.154962,...,2.596004,-0.400534,-11.384759,0.280171,-0.726825,-0.986010,0.005465,0.059020,-0.569002,-0.848452
A1CF,-2.471804,0.652062,0.939450,1.266988,0.569776,0.040014,0.996588,0.571154,-0.172246,0.016433,...,-4.583751,0.163984,-1.499458,0.333793,0.653405,0.291823,-0.451888,-0.309195,-0.774920,-1.303163
A2M,0.794577,0.542774,1.446507,1.404865,0.253899,0.142953,-0.428893,0.050229,1.887457,1.932984,...,-3.080598,-0.046452,-2.274271,-1.105539,1.301845,0.526233,-0.311242,-0.761137,0.284286,2.303708
A2ML1,0.256279,-0.331789,0.873841,0.029529,-0.784894,-0.455278,0.621913,-0.284345,0.487042,-0.125188,...,0.215898,0.898171,1.031624,0.375334,0.377769,-0.031145,0.176213,1.530336,-0.626861,-0.020615
A3GALT2,-1.919669,-0.842670,-1.705009,-1.060155,0.236706,-0.337126,-2.200704,0.288727,-0.012853,-0.432057,...,-1.379109,0.735010,0.225347,-0.580325,-0.748325,-1.537794,0.064722,0.501138,0.000000,0.039885


# Merge Like Column (by taking the mean)

In [44]:
normalized_matrix = uf.merge(normalized_matrix, 'column', 'mean')

In [45]:
normalized_matrix.shape

(19188, 606)

# Map Gene Symbols To Up-to-date Approved Gene Symbols

In [46]:
uf.mapgenesymbols(normalized_matrix)

In [47]:
normalized_matrix.shape

(18757, 606)

# Merge Duplicate Genes By Rows

In [48]:
normalized_matrix = uf.merge(normalized_matrix, 'row', 'mean')

In [49]:
normalized_matrix.shape

(18665, 606)

# Histogram of First Sample

In [50]:
sns.distplot(normalized_matrix.iloc[:, 10]);

# Histogram of First Gene

In [51]:
sns.distplot(normalized_matrix.iloc[10, :]);

# Save Filtered Matrix

In [52]:
filename = path+'tcga_kirc_matrix_filltered_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
normalized_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene List

In [53]:
gene_list = uf.createGeneList(normalized_matrix)

In [54]:
gene_list.head()

,GeneSym,GeneID
0,A1BG,1
1,A1CF,29974
2,A2M,2
3,A2ML1,144568
4,A3GALT2,127550


In [55]:
gene_list.shape

(18665, 2)

# Save Gene List

In [56]:
filename = path+'tcga_kirc_gene_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_list.to_csv(filename, sep='\t', compression='gzip')

# Create Attribute List

In [57]:
metaData = pd.DataFrame(index=normalized_matrix.columns)

for index in metaData.index:
    if ('-').join(index.split('-')[0:3]) in sample_meta.index:
        metaData.loc[index, 'Age'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'age_at_initial_pathologic_diagnosis']
        metaData.loc[index, 'Gender'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'gender']
        metaData.loc[index, 'Pathologic M'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_M']
        metaData.loc[index, 'Pathologic N'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_N']
        metaData.loc[index, 'Pathologic T'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_T']
        metaData.loc[index, 'Pathologic Stage'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_stage']
        metaData.loc[index, 'Histological Type'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'histological_type']
        metaData.loc[index, 'Sample Type'] = SampleTypes[str(index.split('-')[3])]
        metaData.loc[index, 'Tumor Tissue Site'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'tumor_tissue_site']
        metaData.loc[index, 'Neoplasm Histologic Grade'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'neoplasm_histologic_grade']
    else:
        metaData.loc[index, 'Age'] = np.nan
        metaData.loc[index, 'Gender'] = np.nan
        metaData.loc[index, 'Pathologic M'] = np.nan
        metaData.loc[index, 'Pathologic N'] = np.nan
        metaData.loc[index, 'Pathologic T'] = np.nan
        metaData.loc[index, 'Pathologic Stage'] = np.nan
        metaData.loc[index, 'Histological Type'] = np.nan
        metaData.loc[index, 'Sample Type'] = SampleTypes[str(index.split('-')[3])]
        metaData.loc[index, 'Tumor Tissue Site'] = np.nan
        metaData.loc[index, 'Neoplasm Histologic Grade'] = np.nan

In [58]:
attribute_list = uf.createAttributeList(normalized_matrix, metaData)

In [59]:
attribute_list.head()

,Age,Gender,Pathologic M,Pathologic N,Pathologic T,Pathologic Stage,Histological Type,Sample Type,Tumor Tissue Site,Neoplasm Histologic Grade
Attributes,,,,,,,,,,
TCGA-3Z-A93Z-01,69,MALE,M0,N0,T1a,Stage I,Kidney Clear Cell Renal Carcinoma,Primary Solid Tumor,Kidney,G2
TCGA-6D-AA2E-01,68,FEMALE,MX,NX,T1b,Stage I,Kidney Clear Cell Renal Carcinoma,Primary Solid Tumor,Kidney,G2
TCGA-A3-3306-01,67,MALE,M0,N0,T1b,Stage I,Kidney Clear Cell Renal Carcinoma,Primary Solid Tumor,Kidney,G3
TCGA-A3-3307-01,66,MALE,M0,N0,T3b,Stage III,Kidney Clear Cell Renal Carcinoma,Primary Solid Tumor,Kidney,G3
TCGA-A3-3308-01,77,FEMALE,M0,N0,T3b,Stage III,Kidney Clear Cell Renal Carcinoma,Primary Solid Tumor,Kidney,G2


In [60]:
attribute_list.shape

(606, 10)

# Save Attribute List

In [62]:
filename = path+'tcga_kirc_attribute_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_list.to_csv(filename, sep='\t', compression='gzip')

# Create matrix of Standardized values (values between -1, and 1)

In [63]:
standard_matrix = uf.createStandardizedMatrix(normalized_matrix)

In [64]:
standard_matrix.head()

,TCGA-3Z-A93Z-01,TCGA-6D-AA2E-01,TCGA-A3-3306-01,TCGA-A3-3307-01,TCGA-A3-3308-01,TCGA-A3-3311-01,TCGA-A3-3313-01,TCGA-A3-3316-01,TCGA-A3-3317-01,TCGA-A3-3319-01,...,TCGA-G6-A5PC-01,TCGA-G6-A8L6-01,TCGA-G6-A8L7-01,TCGA-G6-A8L8-01,TCGA-GK-A6C7-01,TCGA-MM-A563-01,TCGA-MM-A564-01,TCGA-MM-A84U-01,TCGA-MW-A4EC-01,TCGA-T7-A92I-01
,,,,,,,,,,,,,,,,,,,,,
A1BG,-0.000004,-0.808094,0.485283,-0.438667,-0.633251,-0.089026,0.673553,-0.029704,-0.722314,0.221108,...,0.964404,0.762795,0.931381,0.445617,-0.441968,-0.468393,0.822332,0.653730,0.089144,0.273886
A1CF,-0.122851,-0.591220,0.081712,0.738892,-0.007387,0.633079,-0.531845,-0.258096,-0.535147,0.210369,...,-0.314166,-0.231751,-0.940804,-0.294412,0.712424,-0.380166,-0.686856,-0.624180,-0.112955,-0.835283
A2M,-0.139370,-0.855114,-0.752843,0.286252,0.533975,-0.294433,-0.726460,0.071789,0.527372,-0.868294,...,-0.940825,-0.465912,-0.927644,-0.287849,0.867741,0.249971,-0.103082,-0.182219,-0.545062,-0.944107
A2ML1,-0.453785,0.784045,0.764218,0.034360,0.291803,-0.216332,0.992203,0.093833,0.549424,0.374360,...,-0.321831,0.037628,0.159806,0.605571,0.513072,-0.460367,0.295153,0.027758,-0.457069,0.430461
A3GALT2,0.060175,0.277952,-0.226837,0.634650,0.783403,-0.517091,-0.883173,-0.609414,0.307713,-0.315875,...,-0.616013,0.132795,-0.599522,-0.681984,-0.477503,0.004042,-0.411557,-0.454426,-0.197160,-0.728160


# Plot of A Single Celltype, Normalized Value vs. Standardized Value

In [65]:
plt.plot(normalized_matrix[normalized_matrix.columns[0]], standard_matrix[standard_matrix.columns[0]], 'bo')
plt.xlabel('Normalized Values')
plt.ylabel('Standardized Values')
plt.title(standard_matrix.columns[0])
plt.grid(True)

# Save Standardized Matrix

In [66]:
filename = path+'tcga_kirc_standard_matrix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
standard_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Tertiary Matrix

In [67]:
tertiary_matrix = uf.createTertiaryMatrix(standard_matrix)

In [68]:
tertiary_matrix.head()

,TCGA-3Z-A93Z-01,TCGA-6D-AA2E-01,TCGA-A3-3306-01,TCGA-A3-3307-01,TCGA-A3-3308-01,TCGA-A3-3311-01,TCGA-A3-3313-01,TCGA-A3-3316-01,TCGA-A3-3317-01,TCGA-A3-3319-01,...,TCGA-G6-A5PC-01,TCGA-G6-A8L6-01,TCGA-G6-A8L7-01,TCGA-G6-A8L8-01,TCGA-GK-A6C7-01,TCGA-MM-A563-01,TCGA-MM-A564-01,TCGA-MM-A84U-01,TCGA-MW-A4EC-01,TCGA-T7-A92I-01
,,,,,,,,,,,,,,,,,,,,,
A1BG,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
A1CF,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A2M,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A2ML1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A3GALT2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Save Teriary Matrix

In [69]:
filename = path+'tcga_kirc_tertiary_matrix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
tertiary_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Up Gene Set Library

In [70]:
name = 'tcga_kirc_gene_up_set'

In [71]:
uf.createUpGeneSetLib(tertiary_matrix, path, name)

# Create Down Gene Set Library

In [72]:
name = 'tcga_kirc_gene_down_set'

In [73]:
uf.createDownGeneSetLib(tertiary_matrix, path, name)

# Create Up Attribute Library

In [74]:
name = 'tcga_kirc_attribute_up_set'

In [75]:
uf.createUpAttributeSetLib(tertiary_matrix, path, name)

# Create Down Attribute Library

In [76]:
name = 'tcga_kirc_attribute_down_set'

In [77]:
uf.createDownAttributeSetLib(tertiary_matrix, path, name)

# Create Attribute Similarity matrix

In [78]:
attribute_normalized_matrix = normalized_matrix.copy()

In [79]:
getSampleData(attribute_normalized_matrix, sample_meta, SampleTypes)

In [80]:
attribute_similarity_matix = uf.createSimilarityMatrix(attribute_normalized_matrix.T, 'cosine')

In [81]:
attribute_similarity_matix.head()

,"(Donor:TCGA-3Z-A93Z, Age:69.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage I, Histology:Kidney Clear Cell Renal Carcinoma, Tissue:Kidney, Tumor:nan, GradeG2)","(Donor:TCGA-6D-AA2E, Age:68.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage I, Histology:Kidney Clear Cell Renal Carcinoma, Tissue:Kidney, Tumor:nan, GradeG2)","(Donor:TCGA-A3-3306, Age:67.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage I, Histology:Kidney Clear Cell Renal Carcinoma, Tissue:Kidney, Tumor:nan, GradeG3)","(Donor:TCGA-A3-3307, Age:66.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage III, Histology:Kidney Clear Cell Renal Carcinoma, Tissue:Kidney, Tumor:nan, GradeG3)","(Donor:TCGA-A3-3308, Age:77.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage III, Histology:Kidney Clear Cell Renal Carcinoma, Tissue:Kidney, Tumor:nan, GradeG2)","(Donor:TCGA-A3-3311, Age:57.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage I, Histology:Kidney Clear Cell Renal Carcinoma, Tissue:Kidney, Tumor:nan, GradeG2)","(Donor:TCGA-A3-3313, Age:59.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage I, Histology:Kidney Clear Cell Renal Carcinoma, Tissue:Kidney, Tumor:nan, GradeG3)","(Donor:TCGA-A3-3316, Age:57.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage II, Histology:Kidney Clear Cell Renal Carcinoma, Tissue:Kidney, Tumor:nan, GradeG3)","(Donor:TCGA-A3-3317, Age:67.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage II, Histology:Kidney Clear Cell Renal Carcinoma, Tissue:Kidney, Tumor:nan, GradeG2)","(Donor:TCGA-A3-3319, Age:70.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage I, Histology:Kidney Clear Cell Renal Carcinoma, Tissue:Kidney, Tumor:nan, GradeG2)",...,"(Donor:TCGA-G6-A5PC, Age:54.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage IV, Histology:Kidney Clear Cell Renal Carcinoma, Tissue:Kidney, Tumor:nan, GradeG4)","(Donor:TCGA-G6-A8L6, Age:55.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IV, Histology:Kidney Clear Cell Renal Carcinoma, Tissue:Kidney, Tumor:nan, GradeG3)","(Donor:TCGA-G6-A8L7, Age:81.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage I, Histology:Kidney Clear Cell Renal Carcinoma, Tissue:Kidney, Tumor:nan, GradeG3)","(Donor:TCGA-G6-A8L8, Age:62.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage I, Histology:Kidney Clear Cell Renal Carcinoma, Tissue:Kidney, Tumor:nan, GradeG3)","(Donor:TCGA-GK-A6C7, Age:76.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage I, Histology:Kidney Clear Cell Renal Carcinoma, Tissue:Kidney, Tumor:nan, Gradenan)","(Donor:TCGA-MM-A563, Age:41.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Kidney Clear Cell Renal Carcinoma, Tissue:Kidney, Tumor:nan, GradeG2)","(Donor:TCGA-MM-A564, Age:68.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage II, Histology:Kidney Clear Cell Renal Carcinoma, Tissue:Kidney, Tumor:nan, GradeG2)","(Donor:TCGA-MM-A84U, Age:58.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage I, Histology:Kidney Clear Cell Renal Carcinoma, Tissue:Kidney, Tumor:nan, GradeG2)","(Donor:TCGA-MW-A4EC, Age:72.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage I, Histology:Kidney Clear Cell Renal Carcinoma, Tissue:Kidney, Tumor:nan, GradeG2)","(Donor:TCGA-T7-A92I, Age:47.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage I, Histology:Kidney Clear Cell Renal Carcinoma, Tissue:Kidney, Tumor:nan, GradeG1)"
,,,,,,,,,,,,,,,,,,,,,
"(Donor:TCGA-3Z-A93Z, Age:69.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage I, Histology:Kidney Clear Cell Renal Carcinoma, Tissue:Kidney, Tumor:nan, GradeG2)",1.000000,0.215151,0.017806,0.053912,-0.006559,-0.117755,-0.100800,-0.053046,-0.038267,-0.020592,...,0.216230,0.287120,0.226958,0.322348,0.157985,0.325456,0.239643,0.326123,0.291246,-0.053126
"(Donor:TCGA-6D-AA2E, Age:68.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage I, Histology:Kidney Clear Cell Ren

# Save Attribute Similarity Matrix

In [82]:
filename = path+'tcga_kirc_attribute_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# HeatMap (clustergrammer) of Similarity Matrix

In [83]:
net.load_df(attribute_similarity_matix.iloc[:,:].copy())
# net.filter_N_top('row', rank_type='sum', N_top=300)
net.cluster()
net.widget()

# Create Gene Similarity Matrix

In [84]:
gene_similarity_matix = uf.createSimilarityMatrix(normalized_matrix, 'cosine')

In [85]:
gene_similarity_matix.head()

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
,,,,,,,,,,,,,,,,,,,,,
A1BG,1.000000,0.021247,-0.081491,-0.060130,0.016369,-0.030422,-0.098237,-0.001621,0.070376,0.084889,...,-0.010667,0.047776,-0.031593,-0.001234,-0.038927,-0.026957,-0.000941,-0.110423,-0.112010,-0.019465
A1CF,0.021247,1.000000,0.361943,-0.002456,0.108627,0.286661,0.245312,-0.144669,-0.069936,0.001046,...,-0.393671,0.021903,0.323867,0.083283,0.272897,0.372133,0.156614,-0.073245,0.077380,0.241285
A2M,-0.081491,0.361943,1.000000,-0.060145,-0.003775,0.312796,0.112294,-0.342673,-0.079499,-0.166272,...,-0.217596,-0.115621,0.380285,0.136593,0.151113,-0.014689,0.179221,0.257552,0.349942,0.373520
A2ML1,-0.060130,-0.002456,-0.060145,1.000000,0.196605,0.005041,0.359359,0.051968,-0.274074,-0.030377,...,-0.010533,-0.211686,-0.099465,-0.242203,-0.091442,0.146747,-0.030762,0.103588,0.107600,-0.084988
A3GALT2,0.016369,0.108627,-0.003775,0.196605,1.000000,0.084391,0.172069,0.065533,-0.206335,0.026811,...,-0.080026,-0.035926,-0.100776,-0.149543,-0.085339,0.005889,-0.095452,0.143919,-0.051224,-0.098931


# Save Gene Similarity Matrix

In [86]:
filename = path+'tcga_kirc_gene_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene-Attribute Edge List

In [87]:
name = 'tcga_kirc_gene_attribute_edge_list'

In [88]:
uf.createGeneAttributeEdgeList(standard_matrix, attribute_list, gene_list, path, name)

Progeres: 100%  606 Out of 606   

 The number of statisticaly relevent gene-attribute associations is: 565628
